In [196]:
import numpy as np
import torch
from torch.utils import data
import torch.nn as nn

In [197]:
# parameters
w_data = torch.tensor([2, -3.4])
b_data = 4.2
size_data = 1000
epoch_learn = 3
batch_learn = 10
lr_learn = 0.03

# data
def make_data(w, b, n):
    X = torch.normal(0, 1, (n, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y

X_data, y_data = make_data(w_data, b_data, size_data)
def data_iter(batch, X, y):
    n = len(y)
    index = np.random.permutation(np.arange(n))
    for i in range(0, n, batch):
        ind = index[i:min(i + batch, n)]
        yield X[ind], y[ind]

# model
w_model = torch.normal(0, 0.01, size=[2], requires_grad=True)
b_model = torch.zeros(1, requires_grad=True)
def linear_model(X, w, b):
    return torch.matmul(X, w) + b

def loss(y_hat, y):
    return torch.sum((y_hat - y) ** 2) / 2

def sgd(params, lr, batch):
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch
            param.grad.zero_()

# training
for epoch in range(epoch_learn):
    for X_batch, y_batch in data_iter(batch_learn, X_data, y_data):
        loss(linear_model(X_batch, w_model, b_model), y_batch).backward()
        sgd([w_model, b_model], lr_learn, batch_learn)

    with torch.no_grad():
        loss_learn = loss(linear_model(X_data, w_model, b_model), y_data)
        print('epoch {}, loss {:.4f}'.format(epoch + 1, loss_learn))

# results
print((w_model - w_data).data)
print((b_model - b_data).data)

epoch 1, loss 41.1285
epoch 2, loss 0.1581
epoch 3, loss 0.0502
tensor([-1.9312e-05, -4.9591e-05])
tensor([-0.0007])


In [198]:
# parameters
w_data = torch.tensor([2, -3.4])
b_data = 4.2
size_data = 1000
batch_learn = 10
lr_learn = 0.03
epoch_learn = 3

# data
def make_data(w, b, n):
    X = torch.normal(0, 1, (n, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape([-1, 1])

X_data, y_data = make_data(w_data, b_data, size_data)
def load_array(dataset, batch, train=True):
    dataset = data.TensorDataset(*dataset)
    return data.DataLoader(dataset, batch, shuffle=train)

data_iter = load_array([X_data, y_data], batch_learn)

# model
model = nn.Sequential(nn.Linear(2, 1))
model[0].weight.data.normal_(0, 0.01)
model[0].bias.data.fill_(0)
loss = nn.MSELoss()
trainer = torch.optim.SGD(model.parameters(), lr=lr_learn)

# training
for epoch in range(epoch_learn):
    for X_batch, y_batch in data_iter:
        loss(model(X_batch), y_batch).backward()
        trainer.step()
        trainer.zero_grad()

    loss_learn = loss(model(X_data), y_data)
    print('epoch {}, loss {:.4f}'.format(epoch + 1, loss_learn))

# results
print((model[0].weight - w_data).data)
print((model[0].bias - b_data).data)

epoch 1, loss 0.0004
epoch 2, loss 0.0001
epoch 3, loss 0.0001
tensor([[ 8.4758e-04, -7.6056e-05]])
tensor([0.0002])
